# Generator
Build generator architecture with:
* Attention blocks
* Spectral Convolutions
* Upsampling and Downsampling layers
* Resnet blocks

In [ ]:
import tensorflow as tf

def attention_block(x, filters):
    """Self-attention mechanism with dynamic input support"""
    shape = tf.shape(x)
    batch_size, height, width = shape[0], shape[1], shape[2]
    
    f = tf.keras.layers.Conv2D(filters // 8, 1)(x)
    g = tf.keras.layers.Conv2D(filters // 8, 1)(x)
    h = tf.keras.layers.Conv2D(filters, 1)(x)
    
    f = tf.reshape(f, [batch_size, -1, filters // 8])
    g = tf.reshape(g, [batch_size, -1, filters // 8])
    h = tf.reshape(h, [batch_size, -1, filters])
    
    s = tf.matmul(f, g, transpose_b=True)
    beta = tf.nn.softmax(s)
    
    o = tf.matmul(beta, h)
    o = tf.reshape(o, [batch_size, height, width, filters])
    
    gamma = tf.Variable(0.0, trainable=True)
    return x + gamma * o

def spectral_conv2d(filters, kernel_size, strides=1):
    return tf.keras.layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        padding='same',
        kernel_initializer='orthogonal',
        use_bias=False
    )

def downsample(filters, size, apply_batchnorm=True):
    result = tf.keras.Sequential()
    result.add(spectral_conv2d(filters, size, strides=2))
    
    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization(momentum=0.9))
    
    result.add(tf.keras.layers.PReLU(shared_axes=[1, 2]))
    return result

def upsample(filters, size, apply_dropout=False):
    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv2DTranspose(
            filters, size, strides=2,
            padding='same',
            kernel_initializer='orthogonal',
            use_bias=False
        )
    )
    
    result.add(tf.keras.layers.LayerNormalization())
    
    if apply_dropout:
        result.add(tf.keras.layers.SpatialDropout2D(0.5))
    
    result.add(tf.keras.layers.PReLU(shared_axes=[1, 2]))
    return result

def resnet_block(input_tensor, filters, kernel_size=3):
    def squeeze_excite_block(input_tensor, ratio=16):
        channels = input_tensor.shape[-1]
        
        se = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
        se = tf.keras.layers.Reshape((1, 1, channels))(se)
        se = tf.keras.layers.Dense(channels // ratio, activation='relu')(se)
        se = tf.keras.layers.Dense(channels, activation='sigmoid')(se)
        
        return tf.keras.layers.Multiply()([input_tensor, se])
    
    x = spectral_conv2d(filters, kernel_size)(input_tensor)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.PReLU(shared_axes=[1, 2])(x)
    
    x = spectral_conv2d(filters, kernel_size)(x)
    x = tf.keras.layers.LayerNormalization()(x)
    
    x = squeeze_excite_block(x)
    x = tf.keras.layers.Add()([x, input_tensor])
    x = tf.keras.layers.PReLU(shared_axes=[1, 2])(x)
    
    return x

In [ ]:
import tensorflow as tf

class DynamicPaddingLayer(tf.keras.layers.Layer):
    """Custom layer to handle padding to multiples of n"""
    def __init__(self, target_multiple=8, **kwargs):
        super(DynamicPaddingLayer, self).__init__(**kwargs)
        self.target_multiple = target_multiple
        self.built = False
        
    def build(self, input_shape):
        # Create the variables in build method
        self.pad_top = self.add_weight(
            name='pad_top',
            shape=(),
            dtype=tf.int32,
            trainable=False,
            initializer=tf.keras.initializers.Constant(0)
        )
        self.pad_bottom = self.add_weight(
            name='pad_bottom',
            shape=(),
            dtype=tf.int32,
            trainable=False,
            initializer=tf.keras.initializers.Constant(0)
        )
        self.pad_left = self.add_weight(
            name='pad_left',
            shape=(),
            dtype=tf.int32,
            trainable=False,
            initializer=tf.keras.initializers.Constant(0)
        )
        self.pad_right = self.add_weight(
            name='pad_right',
            shape=(),
            dtype=tf.int32,
            trainable=False,
            initializer=tf.keras.initializers.Constant(0)
        )
        self.built = True
        super(DynamicPaddingLayer, self).build(input_shape)
        
    def call(self, inputs):
        shape = tf.shape(inputs)
        height, width = shape[1], shape[2]
        
        # Calculate required padding
        pad_height = (self.target_multiple - (height % self.target_multiple)) % self.target_multiple
        pad_width = (self.target_multiple - (width % self.target_multiple)) % self.target_multiple
        
        # Calculate padding for each dimension
        pad_top = pad_height // 2
        pad_bottom = pad_height - pad_top
        pad_left = pad_width // 2
        pad_right = pad_width - pad_left
        
        # Update the padding variables
        self.pad_top.assign(pad_top)
        self.pad_bottom.assign(pad_bottom)
        self.pad_left.assign(pad_left)
        self.pad_right.assign(pad_right)
        
        # Pad the tensor
        paddings = [[0, 0], [pad_top, pad_bottom], [pad_left, pad_right], [0, 0]]
        padded_tensor = tf.pad(inputs, paddings, mode='REFLECT')
        
        return padded_tensor
    
    def get_config(self):
        config = super(DynamicPaddingLayer, self).get_config()
        config.update({
            'target_multiple': self.target_multiple
        })
        return config


class UnpadLayer(tf.keras.layers.Layer):
    """Custom layer to remove padding"""
    def __init__(self, padding_layer, **kwargs):
        super(UnpadLayer, self).__init__(**kwargs)
        self.padding_layer = padding_layer
        
    def call(self, inputs):
        shape = tf.shape(inputs)
        height = shape[1] - (self.padding_layer.pad_top + self.padding_layer.pad_bottom)
        width = shape[2] - (self.padding_layer.pad_left + self.padding_layer.pad_right)
        
        return inputs[:, 
                     self.padding_layer.pad_top:self.padding_layer.pad_top + height,
                     self.padding_layer.pad_left:self.padding_layer.pad_left + width,
                     :]
    
    def get_config(self):
        config = super(UnpadLayer, self).get_config()
        config.update({
            'padding_layer': self.padding_layer
        })
        return config


def build_generator(input_shape=(None, None, 3)):
    """Build generator with support for arbitrary input sizes"""
    tf.keras.backend.set_floatx('float32')
    
    # Input layer
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # Add padding layer
    padding_layer = DynamicPaddingLayer(target_multiple=8)
    x = padding_layer(inputs)
    
    # Initial convolution
    x = spectral_conv2d(32, 7)(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.PReLU(shared_axes=[1, 2])(x)
    
    # Downsampling path
    down_stack = [
        downsample(64, 3, apply_batchnorm=False),
        downsample(128, 3),
        downsample(256, 3),
    ]
    
    # Store skip connections
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    
    # ResNet blocks
    num_res_blocks = 9
    for i in range(num_res_blocks):
        x = resnet_block(x, filters=256)
        if i % 2 == 0:
            x = attention_block(x, filters=256)
    
    # Upsampling path
    up_stack = [
        upsample(128, 3, apply_dropout=True),
        upsample(64, 3),
        upsample(32, 3),
    ]
    
    # Decoder with skip connections
    skips = reversed(skips[:-1])
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = attention_block(x, x.shape[-1])
        x = tf.keras.layers.Concatenate()([x, skip])
    
    # Final output layer
    x = tf.keras.layers.Conv2DTranspose(
        3, 4,
        strides=2,
        padding='same',
        kernel_initializer='orthogonal',
        activation='tanh'
    )(x)
    
    # Remove padding using custom unpad layer
    outputs = UnpadLayer(padding_layer)(x)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
import numpy as np

# Test function
def test_arbitrary_generator_input():
    model = build_generator(input_shape=(None, None, 3))
    
    test_sizes = [
        (1, 65, 65, 3),    # Odd dimensions
        (1, 100, 100, 3),  # Non-multiple of 8
        (1, 128, 128, 3),  # Multiple of 8
        (1, 77, 153, 3),   # Different height and width
    ]
    
    for size in test_sizes:
        test_input = tf.random.normal(size)
        output = model(test_input)
        input_hw = size[1:3]
        output_hw = tuple(output.shape[1:3])
        print(f"Input size: {input_hw}")
        print(f"Output size: {output_hw}")
        print(f"Shapes match: {input_hw == output_hw}\n")

if __name__ == "__main__":
    test_arbitrary_generator_input()

In [ ]:
generator = build_generator(input_shape=(None, None, 3))
generator.summary()

# Discriminator
Build discriminator architecture:

In [ ]:
# DISCRIMINATOR
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers

def build_discriminator(input_shape=(None, None, 3)):
    """
    Stabilized PatchGAN discriminator that handles arbitrary input sizes
    """
    # Add spectral normalization
    def conv_block(x, filters, kernel_size=4, strides=2, apply_norm=True):
        x = tfa.layers.SpectralNormalization(
            layers.Conv2D(filters, kernel_size, strides, padding='same',
                         kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01),
                         use_bias=False))(x)
        
        if apply_norm:
            x = tfa.layers.InstanceNormalization()(x)
        
        x = layers.LeakyReLU(alpha=0.2)(x)
        return x

    # Use None for dynamic dimensions
    inp = layers.Input(shape=input_shape, dtype=tf.float32, name='input_image')
    tar = layers.Input(shape=input_shape, dtype=tf.float32, name='target_image')
    
    # Add noise directly to inputs
    noise_scale = tf.Variable(0.005, trainable=True)
    noise = tf.random.normal(tf.shape(inp)) * noise_scale
    x = layers.Concatenate(axis=-1)([inp + noise, tar + noise])
    
    # Network will now work with any input size
    x = conv_block(x, 32, strides=2)        # H/2 x W/2
    x = conv_block(x, 64, strides=2)        # H/4 x W/4
    x = conv_block(x, 128, strides=2)       # H/8 x W/8
    x = conv_block(x, 256, strides=1)       # H/8 x W/8
    
    x = layers.Conv2D(1, 3, strides=1, padding='same',
                     kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)
    
    return tf.keras.Model(inputs=[inp, tar], outputs=x)

In [ ]:
discriminator = build_discriminator()
discriminator.summary()

# Loss function class
An object with physics-informed loss for the generator and discriminator

In [ ]:
import tensorflow as tf

class FluidGANLoss:
    def __init__(self, discriminator, lambda_gp=10.0, lambda_physics=1.0):
        self.discriminator = discriminator
        self.lambda_gp = lambda_gp
        self.lambda_physics = lambda_physics
        # Add adaptive weights
        self.adversarial_weight = tf.Variable(1.0, trainable=False)
        self.running_gen_loss = tf.Variable(0.0, trainable=False)
        self.running_disc_loss = tf.Variable(0.0, trainable=False)
        self.beta = 0.99  # For running average
        
    def generator_loss(self, disc_generated_output, y_true, y_pred):
        # Use least squares GAN loss instead of relativistic
        # This tends to be more stable for fluid dynamics
        adversarial_loss = tf.reduce_mean(tf.square(disc_generated_output - 1.0))
        
        # Enhanced L1 loss with focus on high-gradient regions
        base_l1 = tf.abs(y_true - y_pred)
        grad_weights = tf.reduce_mean(tf.image.sobel_edges(y_true), axis=-1)
        grad_weights = tf.nn.softmax(grad_weights * 10.0)  # Temperature scaling
        weighted_l1 = base_l1 * (1.0 + grad_weights)
        l1_loss = tf.reduce_mean(weighted_l1)
        
        # Gradient matching with different scales
        grad_loss = 0.0
        scales = [1, 2]
        for scale in scales:
            if scale > 1:
                y_true_scaled = tf.nn.avg_pool2d(y_true, scale, scale, 'VALID')
                y_pred_scaled = tf.nn.avg_pool2d(y_pred, scale, scale, 'VALID')
            else:
                y_true_scaled = y_true
                y_pred_scaled = y_pred
                
            true_grads = tf.image.sobel_edges(y_true_scaled)
            pred_grads = tf.image.sobel_edges(y_pred_scaled)
            grad_loss += tf.reduce_mean(tf.abs(true_grads - pred_grads))
        
        # Physics loss with adaptive weight
        physics_loss = self.physics_consistency_loss(y_pred)
        physics_weight = tf.minimum(1.0, tf.cast(self.running_gen_loss, tf.float32) / 2.0)
        
        # Dynamically adjust weights based on running losses
        l1_weight = 50.0  # Reduced from 100.0
        grad_weight = 10.0  # Reduced from 25.0
        
        total_loss = (
            self.adversarial_weight * adversarial_loss + 
            l1_weight * l1_loss +
            grad_weight * grad_loss + 
            physics_weight * self.lambda_physics * physics_loss
        )
        
        # Update running average of generator loss
        self.running_gen_loss.assign(
            self.beta * self.running_gen_loss + (1 - self.beta) * total_loss
        )
        
        return {
            'total_loss': total_loss,
            'adversarial_loss': adversarial_loss,
            'l1_loss': l1_loss,
            'grad_loss': grad_loss,
            'physics_loss': physics_loss,
            'physics_weight': physics_weight
        }
    
    def discriminator_loss(self, disc_real_output, disc_generated_output, 
                          real_images, generated_images, batch_size):
        # Use least squares GAN loss for discriminator
        real_loss = tf.reduce_mean(tf.square(disc_real_output - 1.0))
        fake_loss = tf.reduce_mean(tf.square(disc_generated_output))
        
        # Calculate gradient penalty with adaptive weight
        gp = self.gradient_penalty(real_images, generated_images, batch_size)
        gp_weight = tf.minimum(10.0, 2.0 * tf.cast(self.running_disc_loss, tf.float32))
        
        total_loss = real_loss + fake_loss + gp_weight * gp
        
        # Update running average of discriminator loss
        self.running_disc_loss.assign(
            self.beta * self.running_disc_loss + (1 - self.beta) * total_loss
        )
        
        # Adjust adversarial weight for generator based on discriminator performance
        disc_ratio = real_loss / (fake_loss + 1e-6)
        self.adversarial_weight.assign(
            tf.clip_by_value(self.adversarial_weight * tf.sqrt(disc_ratio), 0.5, 2.0)
        )
        
        return {
            'total_loss': total_loss,
            'real_loss': real_loss,
            'fake_loss': fake_loss,
            'gradient_penalty': gp,
            'gp_weight': gp_weight
        }
    
    def physics_consistency_loss(self, y_pred):
        """Physics-based loss focusing on fluid dynamics constraints"""
        # Calculate velocity gradients using central differences
        dx = (y_pred[:, 2:, 1:-1, :] - y_pred[:, :-2, 1:-1, :]) / 2.0
        dy = (y_pred[:, 1:-1, 2:, :] - y_pred[:, 1:-1, :-2, :]) / 2.0
        
        # Divergence-free constraint
        divergence = dx + dy
        divergence_loss = tf.reduce_mean(tf.square(divergence))
        
        # Add smoothness constraint for better stability
        smoothness_loss = tf.reduce_mean(tf.square(dx)) + tf.reduce_mean(tf.square(dy))
        
        # Boundary conditions
        boundary_loss = tf.reduce_mean(tf.square(y_pred[:, 0, :, :]) + 
                                     tf.square(y_pred[:, -1, :, :]) +
                                     tf.square(y_pred[:, :, 0, :]) + 
                                     tf.square(y_pred[:, :, -1, :]))
        
        return divergence_loss + 0.1 * smoothness_loss + 0.05 * boundary_loss
    
    def gradient_penalty(self, real_images, fake_images, batch_size):
        """Calculate gradient penalty with improved interpolation"""
        # Use random interpolation points
        alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff
        
        with tf.GradientTape() as tape:
            tape.watch(interpolated)
            disc_interpolated = self.discriminator([interpolated, real_images], 
                                                 training=True)
        
        gradients = tape.gradient(disc_interpolated, interpolated)
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2, 3]) + 1e-8)
        return tf.reduce_mean(tf.square(slopes - 1.0))

# Training setup
An object containing all the necessary capabilities to train the model and preview progress

In [ ]:
import matplotlib.pyplot as plt

def generate_images(model, test_input, tar):
    """ Generates and displays a comparison between input image, 
    ground truth and predicted image from a model.
    """
    # Remove any extra batch dimensions if present
    if len(test_input.shape) > 4:
        test_input = tf.squeeze(test_input, axis=0)
    if len(tar.shape) > 4:
        tar = tf.squeeze(tar, axis=0)
    
    # Add batch dimension if needed
    if len(test_input.shape) == 3:
        test_input = tf.expand_dims(test_input, 0)
    if len(tar.shape) == 3:
        tar = tf.expand_dims(tar, 0)
    
    prediction = model(test_input, training=True)
    plt.figure(figsize=(10, 10))
    value_range = .05
    
    display_list = [
        test_input[0][:,:,0], tar[0][:,:,0], prediction[0][:,:,0],
        test_input[0][:,:,1], tar[0][:,:,1], prediction[0][:,:,1],
        test_input[0][:,:,2], tar[0][:,:,2], prediction[0][:,:,2] 
    ]
    
    maintitle = ['Input Image', 'Ground Truth', 'Predicted Image']
    title = ['vel.x', 'vel.y', 'pressure']
    
    for i in range(9):
        plt_idx = i
        plt.subplot(3, 3, i+1)
        
        try: 
            header = f"----- {maintitle[i]} -----"
        except: 
            header = ""
        plt.title(f"{header}\n\n{title[i//3]}", fontsize=11)
        
        # Getting the pixel values in the [0, 1] range to plot.
        tile = display_list[i]
        plt.imshow(tile, vmin=-value_range, vmax=value_range)
        plt.axis('off')
    plt.show()

In [ ]:
import tensorflow as tf

class DataAugmentor:
    def __init__(self, min_size=64, max_size=104):
        self.min_size = min_size
        self.max_size = max_size

    @tf.function
    def augment(self, input_image, target_image):
        """Apply consistent augmentations to both input and target"""

        # Generate random parameters once to apply same transformation to both
        do_flip_lr = tf.random.uniform([]) > 0.5
        do_flip_ud = tf.random.uniform([]) > 0.5
        rotation_factor = tf.random.uniform([], 0, 4, dtype=tf.int32)

        height = tf.random.uniform([], self.min_size, self.max_size, dtype=tf.int32)
        width = tf.random.uniform([], self.min_size, self.max_size, dtype=tf.int32)

        # Apply same transformations to both images
        for image in [input_image, target_image]:
            if do_flip_lr:
                image = tf.image.flip_left_right(image)
            if do_flip_ud:
                image = tf.image.flip_up_down(image)
            image = tf.image.rot90(image, k=rotation_factor)
            image = tf.image.resize(image, [height, width])
            image = tf.image.resize(image, [64, 64])  # Final consistent size

        return input_image, target_image

In [ ]:
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import datetime
import numpy as np

class FluidGANTrainer:
    def __init__(
        self,
        generator,
        discriminator,
        train_dataset,
        test_dataset,
        checkpoint_dir='./training_checkpoints',
        preview_interval=10,
        checkpoint_interval=50,
        validation_interval=5,
        reaugment_interval=5,  # Re-augment every N epochs
        augmentation_factor=2,  # Number of augmented versions per original sample
        samples_per_epoch=3000,
        batch_size=10,
        log_rate=200, # log every N batches
        cache_dir='/tmp/tf_cache'
    ):
        # Existing initialization code
        self.generator = generator
        self.discriminator = discriminator
        self.raw_train_dataset = train_dataset
        self.checkpoint_dir = checkpoint_dir
        self.preview_interval = preview_interval
        self.checkpoint_interval = checkpoint_interval
        self.validation_interval = validation_interval
        self.reaugment_interval = reaugment_interval
        self.augmentation_factor = augmentation_factor
        self.samples_per_epoch = samples_per_epoch
        self.batch_size = batch_size
        self.log_rate = log_rate
        self.cache_dir = cache_dir
        
        # Initialize data augmentor
        self.augmentor = DataAugmentor()
        
        # Setup augmented datasets
        self.train_dataset = None  # Will be initialized in setup_training_dataset
        self.test_dataset = self._setup_augmented_dataset(test_dataset, is_training=False)
        
        # Add learning rate scheduling
        initial_lr_gen = 5e-5 * batch_size
        initial_lr_disc = 1e-5 * batch_size
        self.generator_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_lr_gen, decay_steps=2000, decay_rate=0.98, staircase=True)
        self.discriminator_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_lr_disc, decay_steps=2000, decay_rate=0.98, staircase=True)
        
        # Initialize optimizers
        self.generator_optimizer = tf.keras.optimizers.Adam(
            self.generator_scheduler, beta_1=0.5, beta_2=0.9)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(
            self.discriminator_scheduler, beta_1=0.0, beta_2=0.9, epsilon=1e-8)
        
        # Initialize loss function
        self.loss_fn = FluidGANLoss(discriminator=self.discriminator, lambda_gp=10.0, lambda_physics=0.0)
        
        # Setup checkpointing
        self.checkpoint = tf.train.Checkpoint(
            generator_optimizer=self.generator_optimizer,
            discriminator_optimizer=self.discriminator_optimizer,
            generator=self.generator,
            discriminator=self.discriminator
        )
        self.checkpoint_manager = tf.train.CheckpointManager(
            self.checkpoint, self.checkpoint_dir, max_to_keep=10
        )
        
        # Setup logging
        current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        self.log_dir = f'logs/{current_time}'
        self.summary_writer = tf.summary.create_file_writer(self.log_dir)

    def _setup_augmented_dataset(self, dataset, is_training=True):
        """Setup cached and augmented dataset"""
        if not isinstance(dataset, tf.data.Dataset):
            dataset = tf.data.Dataset.from_tensor_slices(dataset)
        
        if is_training:
            # For training: repeat -> shuffle -> augment -> batch -> cache -> prefetch
            pipeline = (dataset
                .repeat(self.augmentation_factor)  # Repeat dataset to allow for more augmentations
                .shuffle(1000 * self.augmentation_factor)  # Increase shuffle buffer for larger dataset
                .map(self.augmentor.augment, 
                     num_parallel_calls=tf.data.AUTOTUNE)
                .batch(self.batch_size)
                .cache()  # Cache after batching
                .prefetch(tf.data.AUTOTUNE))
        else:
            # For testing: batch -> cache -> prefetch
            pipeline = (dataset
                .batch(self.batch_size)
                .cache()  # Cache after batching
                .prefetch(tf.data.AUTOTUNE))
        
        return pipeline
    
    def setup_training_dataset(self, epoch):
        """Setup or refresh training dataset with new augmentations"""
        cache_path = f"{self.cache_dir}/train_epoch_{epoch // self.reaugment_interval}"
        
        # Clear previous cached datasets if they exist
        try:
            tf.io.gfile.rmtree(f"{self.cache_dir}/train_epoch_{(epoch // self.reaugment_interval) - 1}")
        except:
            pass
            
        print(f"Setting up new augmented dataset for epochs {epoch+1}-{epoch+1 + self.reaugment_interval - 1}")
        print(f"Creating augmented dataset with {self.augmentation_factor}x augmentation "
              f"(from {len(self.raw_train_dataset)} to {len(self.raw_train_dataset) * self.augmentation_factor} samples)")
        
        self.train_dataset = self._setup_augmented_dataset(
            self.raw_train_dataset, 
            is_training=True
        )
    
    def get_epoch_dataset(self, num_samples):
        """Get samples for one epoch"""
        # Calculate actual number of batches needed
        total_samples = len(self.raw_train_dataset) * self.augmentation_factor
        num_batches = min(num_samples, total_samples) // self.batch_size
        
        # Since dataset is already batched and cached, we just need to shuffle and take
        return (self.train_dataset
            .shuffle(num_batches)  # Shuffle at batch level
            .take(num_batches)
            .prefetch(tf.data.AUTOTUNE))
    

    @tf.function
    def train_step(self, input_image, target_image):
        """Single training step"""
        batch_size = tf.shape(input_image)[0]
        
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            # Generator forward pass
            gen_output = self.generator(input_image, training=True)
            
            # Discriminator forward passes
            disc_real_output = self.discriminator([input_image, target_image], training=True)
            disc_generated_output = self.discriminator([input_image, gen_output], training=True)
            
            # Calculate losses
            gen_losses = self.loss_fn.generator_loss(
                disc_generated_output, target_image, gen_output
            )
            disc_losses = self.loss_fn.discriminator_loss(
                disc_real_output, disc_generated_output,
                target_image, gen_output, batch_size
            )
        
        # Calculate gradients
        generator_gradients = gen_tape.gradient(
            gen_losses['total_loss'],
            self.generator.trainable_variables
        )
        discriminator_gradients = disc_tape.gradient(
            disc_losses['total_loss'],
            self.discriminator.trainable_variables
        )
        
        # Apply gradient clipping
        generator_gradients = [tf.clip_by_norm(g, 1.0) for g in generator_gradients]
        discriminator_gradients = [tf.clip_by_norm(g, 1.0) for g in discriminator_gradients]
        
        # Apply gradients
        self.generator_optimizer.apply_gradients(
            zip(generator_gradients, self.generator.trainable_variables)
        )
        self.discriminator_optimizer.apply_gradients(
            zip(discriminator_gradients, self.discriminator.trainable_variables)
        )
        
        return gen_losses, disc_losses

    
    @tf.function
    def validation_step(self, input_image, target_image):
        """Single validation step without gradient updates"""
        batch_size = tf.shape(input_image)[0]
        
        # Generator forward pass
        gen_output = self.generator(input_image, training=False)
        
        # Discriminator forward passes
        disc_real_output = self.discriminator([input_image, target_image], training=False)
        disc_generated_output = self.discriminator([input_image, gen_output], training=False)
        
        # Calculate losses
        gen_losses = self.loss_fn.generator_loss(
            disc_generated_output, target_image, gen_output
        )
        disc_losses = self.loss_fn.discriminator_loss(
            disc_real_output, disc_generated_output,
            target_image, gen_output, batch_size
        )
        
        return gen_losses, disc_losses

    def validate_model(self):
        """Run validation on the test dataset"""
        total_gen_loss = 0
        total_disc_loss = 0
        num_batches = 0
        
        for input_image, target_image in self.test_dataset:
            gen_losses, disc_losses = self.validation_step(input_image, target_image)
            total_gen_loss += gen_losses['total_loss']
            total_disc_loss += disc_losses['total_loss']
            num_batches += 1
        
        avg_gen_loss = total_gen_loss / num_batches
        avg_disc_loss = total_disc_loss / num_batches
        
        return {
            'generator_loss': avg_gen_loss,
            'discriminator_loss': avg_disc_loss
        }

    def train(self, epochs):
        """Main training loop with periodic re-augmentation"""
        start = time.time()
        
        # Try to restore latest checkpoint
        if self.checkpoint_manager.latest_checkpoint:
            self.checkpoint.restore(self.checkpoint_manager.latest_checkpoint)
            print(f'Latest checkpoint restored from {self.checkpoint_manager.latest_checkpoint}')
        
        test_input, test_target = next(iter(self.test_dataset.batch(1)))
        
        for epoch in range(epochs):
            # Re-augment dataset every reaugment_interval epochs
            if epoch % self.reaugment_interval == 0:
                self.setup_training_dataset(epoch)
            
            start_time = time.time()
            
            # Get batched dataset for this epoch
            epoch_dataset = self.get_epoch_dataset(self.samples_per_epoch)
            
            # Train on batches
            for batch_idx, (input_image, target_image) in enumerate(epoch_dataset):
                gen_losses, disc_losses = self.train_step(input_image, target_image)
                
                # Log every @log_rate batches
                if batch_idx % self.log_rate == 0:
                    print(f'Epoch {epoch+1}, Batch {batch_idx}:')
                    gen_lr = self.generator_scheduler(self.generator_optimizer.iterations)
                    disc_lr = self.discriminator_scheduler(self.discriminator_optimizer.iterations)
                    print(f'Generator Loss: {gen_losses["total_loss"]:.4f},  LR: {gen_lr:.2e}')
                    print(f'Discriminator Loss: {disc_losses["total_loss"]:.4f},  LR: {disc_lr:.2e}')
                    
                    # Log to TensorBoard
                    with self.summary_writer.as_default():
                        step = epoch * (self.samples_per_epoch // self.batch_size) + batch_idx
                        tf.summary.scalar('train/gen_total_loss', gen_losses['total_loss'], step=step)
                        tf.summary.scalar('train/gen_adversarial_loss', gen_losses['adversarial_loss'], step=step)
                        tf.summary.scalar('train/l1_loss', gen_losses['l1_loss'], step=step)
                        tf.summary.scalar('train/gen_physics_loss', gen_losses['physics_loss'], step=step)
                        tf.summary.scalar('train/disc_total_loss', disc_losses['total_loss'], step=step)

            
            # Run validation
            if (epoch + 1) % self.validation_interval == 0:
                val_losses = self.validate_model()
                print(f'Validation Losses:')
                print(f'Generator Loss: {val_losses["generator_loss"]:.4f}')
                print(f'Discriminator Loss: {val_losses["discriminator_loss"]:.4f}')
                
                # Log validation metrics
                with self.summary_writer.as_default():
                    tf.summary.scalar('validation/gen_loss', val_losses['generator_loss'], step=(epoch + 1))
                    tf.summary.scalar('validation/disc_loss', val_losses['discriminator_loss'], step=(epoch + 1))
            
            # Preview images
            if (epoch + 1) % self.preview_interval == 0:
                generate_images(self.generator, test_input, test_target)
            
            # Save checkpoint
            if (epoch + 1) % self.checkpoint_interval == 0:
                self.checkpoint_manager.save()
                print(f'Checkpoint saved for epoch {epoch+1}')
            
            print(f'Epoch {epoch+1} took {time.time()-start_time:.2f} seconds')
        
        print(f'Training took {time.time()-start} seconds')

# Ingest Data, build datasets, run training
Load concatenated data output by "pressure_predict_1_data_prep"

In [ ]:
import numpy as np
import time

start_time = time.perf_counter()
# combined_data_path = "./data2D/loaded_data_v2_rand.npz" # Original approach with pre-augmented data
combined_data_path = "./data2D/loaded_data_v2.npz" # Original dataset with ~1600 datapoints of 111x111 size
with np.load(combined_data_path) as data:
    X_train = data["X_train"]
    Y_train = data["Y_train"]
    X_test  = data["X_test"]
    Y_test  = data["Y_test"]

end_time = time.perf_counter()
print(f"Loaded combined array in {end_time - start_time:.2f} seconds.")
print("Loaded X shape: ", np.array(X_train).shape, np.array(X_test).shape)
print("Loaded Y shape: ", np.array(Y_train).shape, np.array(Y_test).shape)

# Normally returns:
# Loaded X shape:  (5000, 64, 64, 3) (1000, 64, 64, 3)
# Loaded Y shape:  (5000, 64, 64, 3) (1000, 64, 64, 3)

In [ ]:
# Test untrained generator and preview.
prev_idx = 10
generate_images(generator, X_train[prev_idx][tf.newaxis, ...], Y_train[prev_idx][tf.newaxis, ...])

In [ ]:
# APPLY BATCHING (with 1)
# ... and achieve output dimensionality of:     [2,     n, 64, 64, 3] 
# ... with desired iteration dimensionality of:     [None, 64, 64, 3]
import tensorflow as tf

X_train = tf.cast(X_train, tf.float32)
Y_train = tf.cast(Y_train, tf.float32)
X_test = tf.cast(X_test, tf.float32)
Y_test = tf.cast(Y_test, tf.float32)

# Normalize data to [-1, 1] range if not already done
# def normalize(x):
#     return tf.clip_by_value(x, -1, 1)

# X_train, Y_train = normalize(X_train), normalize(Y_train)
# X_test, Y_test = normalize(X_test), normalize(Y_test)

# Create datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_ds = train_ds.shuffle(5000)
#train_ds = train_ds.batch(5)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
test_ds = test_ds.shuffle(1000)
#test_ds = test_ds.batch(5)


In [ ]:
import random
log_dir = "./logs"

%load_ext tensorboard
%reload_ext tensorboard
random_port = random.randrange(1001,9900)
print(f"Loading tensorboard on port: {random_port}")
%tensorboard --logdir={log_dir} --reload_multifile=true --port={random_port} --host localhost

In [ ]:
# Initialize models
generator = build_generator()
discriminator = build_discriminator()

# Initialize trainer
trainer = FluidGANTrainer(
    generator=generator,
    discriminator=discriminator,
    train_dataset=train_ds,
    test_dataset=test_ds,
    checkpoint_dir='./training_checkpointsH',
    preview_interval=10,
    checkpoint_interval=20,
    validation_interval=10,
    reaugment_interval=10,
    augmentation_factor=2,
    samples_per_epoch=3000,
    batch_size=10,
    log_rate=100
)

# Start training
trainer.train(epochs=250)

In [ ]:
import netron
model = "./models/pressure_predict_D_v0.h5"
netron.start(model)